In [107]:
#imports
import torch
import numpy as np
import pandas as pd
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
import time
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import datetime
import copy
import os
import pickle
from tqdm import tqdm
from models import vgg

# setting

In [112]:
#mean and std of cifar100 dataset
CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)

BATCH_SIZE = 16
WARM = False # typically used in new training

DATE_FORMAT = '%A_%d_%B_%Y_%Hh_%Mm_%Ss'
#time of we run the script
TIME_NOW = datetime.datetime.now().strftime(DATE_FORMAT)
#data settings
subset = False #for local running
k = 10 #number of samples needed to each class in validation set, because we need to split train and validation

#model settings
USE_TENSORBOARD = False
if USE_TENSORBOARD:
    foo = SummaryWriter()
use_gpu = True

#lr scheduler
BASE_LR = 0.001
EPOCH_DECAY = 4
DECAY_WEIGHT = 0.5

DEVICE = 'cpu'
if use_gpu and torch.cuda.is_available():
    DEVICE = 'cuda'

In [3]:
#read files
def unpickle(file):
    
    with open(file, 'rb') as fo:
        dictionary = pickle.load(fo, encoding='bytes')
    return dictionary

In [11]:
def compute_mean_std(cifar100_dataset):
    """compute the mean and std of cifar100 dataset
    Args:
        cifar100_training_dataset or cifar100_test_dataset
        witch derived from class torch.utils.data

    Returns:
        a tuple contains mean, std value of entire dataset
    """

    data_r = numpy.dstack([cifar100_dataset[i][1][:, :, 0] for i in range(len(cifar100_dataset))])
    data_g = numpy.dstack([cifar100_dataset[i][1][:, :, 1] for i in range(len(cifar100_dataset))])
    data_b = numpy.dstack([cifar100_dataset[i][1][:, :, 2] for i in range(len(cifar100_dataset))])
    mean = numpy.mean(data_r), numpy.mean(data_g), numpy.mean(data_b)
    std = numpy.std(data_r), numpy.std(data_g), numpy.std(data_b)

    return mean, std

In [12]:
#data processing
def reshape_images(data_dict):
    reshaped = data_dict.numpy().reshape(len(data_dict), 1024, 3, order = 'F').reshape(len(data_dict), 32,32,3)
    reshaped_processed = torch.from_numpy(reshaped).float().permute(0, 3, 1, 2)
    return reshaped_processed

In [42]:
def get_training_dataloader(mean, std, batch_size=16, num_workers=2, shuffle=True):
    """ return training dataloader
    Args:
        mean: mean of cifar100 training dataset
        std: std of cifar100 training dataset
        path: path to cifar100 training python dataset
        batch_size: dataloader batchsize
        num_workers: dataloader num_works
        shuffle: whether to shuffle
    Returns: train_data_loader:torch dataloader object
    """

    transform_train = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    #cifar100_training = CIFAR100Train(path, transform=transform_train)
    cifar100_training = torchvision.datasets.CIFAR100(root='', train=True, download=True, transform=transform_train)
    
    try:
        random_index = pickle.load(open("random_index.pkl", 'rb'))
    except:
        random_index = np.random.permutation([i for i in range(50000)])
        pickle.dump(random_index, open("random_index.pkl", 'wb'))
    
    
    train_dataset = torch.utils.data.Subset(cifar100_training, train_index)
    validation_dataset = torch.utils.data.Subset(cifar100_training, validation_index)
    
    cifar100_training_loader = DataLoader(
        train_dataset, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)
    
    cifar100_validation_loader = DataLoader(
        validation_dataset, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)

    return cifar100_training_loader, cifar100_validation_loader

In [43]:
def get_test_dataloader(mean, std, batch_size=16, num_workers=2, shuffle=True):
    """ return training dataloader
    Args:
        mean: mean of cifar100 test dataset
        std: std of cifar100 test dataset
        path: path to cifar100 test python dataset
        batch_size: dataloader batchsize
        num_workers: dataloader num_works
        shuffle: whether to shuffle
    Returns: cifar100_test_loader:torch dataloader object
    """

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    #cifar100_test = CIFAR100Test(path, transform=transform_test)
    cifar100_test = torchvision.datasets.CIFAR100(root='', train=False, download=True, transform=transform_test)
    cifar100_test_loader = DataLoader(
        cifar100_test, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)

    return cifar100_test_loader

In [207]:
class ClipWeightCallBack():
    
    def __init__(self):
        self.zeros_mask = None
    
    # on batch begin
    def get_zeros_mask(self, model):
        
        self.zeros_mask= []

        for weights_matrix in model.parameters():
            self.zeros_mask.append(torch.where(weights_matrix == 0, \
                                     torch.zeros(weights_matrix.data.shape).to(DEVICE), \
                                     torch.ones(weights_matrix.data.shape).to(DEVICE)))
    # on batch end
    def apply_zeros_mask(self, model):
        
        for index, weights_matrix in enumerate(model.parameters()):
            weights_matrix.data = weights_matrix.data * self.zeros_mask[index].to(DEVICE)

In [203]:
def train(model, epoch, train_dataloader, optimizer, loss_function, callbacks = None):

    start = time.time()
    model.to(DEVICE)
    model.train()
    # keep track of the zero mask
    if callbacks != None:
        callbacks.get_zeros_mask(model)
    
    for batch_index, (images, labels) in enumerate(train_dataloader):
        if epoch <= WARM:
            warmup_scheduler.step()

        if use_gpu:
            labels = labels.to(DEVICE)
            images = images.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        if callbacks != None:
            callbacks.apply_zeros_mask(model)
            
        n_iter = (epoch - 1) * len(train_dataloader) + batch_index + 1

        last_layer = list(model.children())[-1]
        for name, para in last_layer.named_parameters():
            if 'weight' in name:
                writer.add_scalar('LastLayerGradients/grad_norm2_weights', para.grad.norm(), n_iter)
            if 'bias' in name:
                writer.add_scalar('LastLayerGradients/grad_norm2_bias', para.grad.norm(), n_iter)

        print('Training Epoch: {epoch} [{trained_samples}/{total_samples}]\tLoss: {:0.4f}\tLR: {:0.6f}'.format(
            loss.item(),
            optimizer.param_groups[0]['lr'],
            epoch=epoch,
            trained_samples=batch_index * BATCH_SIZE + len(images),
            total_samples=len(train_dataloader.dataset)
        ))

        #update training loss for each iteration
        writer.add_scalar('Train/loss', loss.item(), n_iter)

    for name, param in model.named_parameters():
        layer, attr = os.path.splitext(name)
        attr = attr[1:]
        writer.add_histogram("{}/{}".format(layer, attr), param, epoch)

    finish = time.time()

    print('epoch {} training time consumed: {:.2f}s'.format(epoch, finish - start))

In [92]:
def evaluate_model(model, val_dataloader):
    # for validation set or testing set
    start = time.time()
    model.to(DEVICE)
    model.eval()
    
    total_preds = 0
    total_corrects = 0
    
    for batch_index, (images, labels) in enumerate(val_dataloader):
        if use_gpu:
            # labels = labels.to(DEVICE)
            images = images.to(DEVICE)
            
        outputs = model(images)
        _, preds = torch.max(outputs.data, 1)
        
        
        total_preds += len(labels)
        total_corrects += np.sum(preds.cpu().numpy() == labels.numpy())
    
    print("Accuracy is {:.5f}".format(total_corrects/total_preds))
    
    return total_corrects/total_preds

In [53]:
cifar100_training_loader, cifar100_validation_loader = get_training_dataloader(
    CIFAR100_TRAIN_MEAN,
    CIFAR100_TRAIN_STD,
    num_workers = 4,
    batch_size = BATCH_SIZE,
    shuffle = True
)

cifar100_test_loader = get_test_dataloader(
    CIFAR100_TRAIN_MEAN,
    CIFAR100_TRAIN_STD,
    num_workers = 4,
    batch_size = BATCH_SIZE,
    shuffle = True
)

Files already downloaded and verified
Files already downloaded and verified


In [214]:
def load_vgg():
    model = vgg.vgg16_bn()
    weights = torch.load("checkpoints/vgg16-197-best.pth")
    model.load_state_dict(weights)
    model.to(DEVICE)
    
    return model

In [215]:
model = load_vgg()

In [117]:
evaluate_model(model, cifar100_test_loader)

Accuracy is 0.10350


0.1035

In [114]:
writer = SummaryWriter(log_dir=os.path.join(
            'logs', 'vgg', TIME_NOW))

Traceback (most recent call last):
  File "C:\Users\1\Anaconda3\envs\python3.5\lib\multiprocessing\queues.py", line 230, in _feed
    close()
  File "C:\Users\1\Anaconda3\envs\python3.5\lib\multiprocessing\connection.py", line 177, in close
    self._close()
  File "C:\Users\1\Anaconda3\envs\python3.5\lib\multiprocessing\connection.py", line 277, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] 句柄无效。


INFO:tensorflow:Using local port 19288
INFO:tensorflow:Using local port 18335
INFO:tensorflow:Using local port 17170
INFO:tensorflow:Using local port 21253
INFO:tensorflow:Using local port 24034
INFO:tensorflow:Using local port 17593
INFO:tensorflow:Using local port 20321
INFO:tensorflow:Using local port 16590
INFO:tensorflow:Using local port 21716
INFO:tensorflow:Using local port 17480


In [119]:
sgd_optimizer = optim.SGD(model.parameters(), lr= 0.0000001, momentum=0.9, weight_decay=5e-4)
crossEntropyLoss_function = nn.CrossEntropyLoss()

In [124]:
train(model, train_dataloader = cifar100_training_loader, epoch = 10, optimizer = sgd_optimizer, \
                                                          loss_function = crossEntropyLoss_function)

Training Epoch: 10 [16/45000]	Loss: 0.6150	LR: 0.000000
Training Epoch: 10 [32/45000]	Loss: 0.0147	LR: 0.000000
Training Epoch: 10 [48/45000]	Loss: 0.9975	LR: 0.000000
Training Epoch: 10 [64/45000]	Loss: 0.0630	LR: 0.000000
Training Epoch: 10 [80/45000]	Loss: 0.3064	LR: 0.000000
Training Epoch: 10 [96/45000]	Loss: 0.4618	LR: 0.000000
Training Epoch: 10 [112/45000]	Loss: 0.2743	LR: 0.000000
Training Epoch: 10 [128/45000]	Loss: 0.0054	LR: 0.000000
Training Epoch: 10 [144/45000]	Loss: 0.0624	LR: 0.000000
Training Epoch: 10 [160/45000]	Loss: 0.0349	LR: 0.000000
Training Epoch: 10 [176/45000]	Loss: 0.0556	LR: 0.000000
Training Epoch: 10 [192/45000]	Loss: 0.0627	LR: 0.000000
Training Epoch: 10 [208/45000]	Loss: 0.2487	LR: 0.000000
Training Epoch: 10 [224/45000]	Loss: 0.4779	LR: 0.000000
Training Epoch: 10 [240/45000]	Loss: 0.5389	LR: 0.000000
Training Epoch: 10 [256/45000]	Loss: 0.1566	LR: 0.000000
Training Epoch: 10 [272/45000]	Loss: 0.0897	LR: 0.000000
Training Epoch: 10 [288/45000]	Loss: 

KeyboardInterrupt: 

In [138]:
def model_report(model, dataloader):
    # local final score on validation data
    num_zeros = sum([(i.detach().cpu().numpy() == 0).sum() for i in model.parameters()])
    total_parameters = sum([np.prod(i.shape) for i in model.parameters()])
    accuracy = evaluate_model(model, dataloader)
    result = (accuracy + num_zeros/total_parameters)/2
    print("num_zeros / total_parameters ratio is ", num_zeros/total_parameters)
    print("accuracy is ", accuracy)
    print("overall score is ", result)
    return num_zeros/total_parameters, accuracy

In [200]:
def prune_network(model, threshold = 0.01):
    
    # vgg has classifier and features
    for weights_matrix in model.parameters():
        weights_matrix.data = torch.where(torch.abs(weights_matrix.data) >= threshold, \
                                          weights_matrix.data, torch.zeros(weights_matrix.data.shape).to(DEVICE))

In [217]:
def finetune_prune(model, rounds, epoches, train_dataloader, val_dataloader, lr = 0.00001, threshold = 0.001):
    print("Model performance at the beginning...")
    model_report(model, val_dataloader)
    sgd_optimizer = optim.SGD(model.parameters(), lr = lr, momentum = 0.9, weight_decay = 5e-4)
    crossEntropyLoss_function = nn.CrossEntropyLoss()
    prune_callback = ClipWeightCallBack()
    
    print("Start pruning..")
    for i in range(rounds):
        print("Round {}/{}:".format(i + 1, rounds))
        prune_network(model)
        train(model, epoches, train_dataloader, sgd_optimizer, crossEntropyLoss_function, callbacks = prune_callback)
        model_report(model, val_dataloader)
    
    print("Done pruning.")

In [220]:
model = load_vgg()

In [221]:
finetune_prune(model, 3, 4, cifar100_training_loader, cifar100_validation_loader, lr = 0.00001, threshold = 0.01)

Model performance at the beginning...
Accuracy is 0.99760
num_zeros / total_parameters ratio is  0.0
accuracy is  0.9976
overall score is  0.4988
Start pruning..
Round 1/3:
Training Epoch: 4 [16/45000]	Loss: 3.7904	LR: 0.000010
Training Epoch: 4 [32/45000]	Loss: 3.8272	LR: 0.000010
Training Epoch: 4 [48/45000]	Loss: 3.9181	LR: 0.000010
Training Epoch: 4 [64/45000]	Loss: 3.7499	LR: 0.000010
Training Epoch: 4 [80/45000]	Loss: 3.7629	LR: 0.000010
Training Epoch: 4 [96/45000]	Loss: 3.7529	LR: 0.000010
Training Epoch: 4 [112/45000]	Loss: 3.7358	LR: 0.000010
Training Epoch: 4 [128/45000]	Loss: 3.6645	LR: 0.000010
Training Epoch: 4 [144/45000]	Loss: 3.8374	LR: 0.000010
Training Epoch: 4 [160/45000]	Loss: 3.7710	LR: 0.000010
Training Epoch: 4 [176/45000]	Loss: 3.7711	LR: 0.000010
Training Epoch: 4 [192/45000]	Loss: 3.8007	LR: 0.000010
Training Epoch: 4 [208/45000]	Loss: 3.6681	LR: 0.000010
Training Epoch: 4 [224/45000]	Loss: 3.7318	LR: 0.000010
Training Epoch: 4 [240/45000]	Loss: 3.7945	LR: 0.

Training Epoch: 4 [2272/45000]	Loss: 3.7129	LR: 0.000010
Training Epoch: 4 [2288/45000]	Loss: 3.6547	LR: 0.000010
Training Epoch: 4 [2304/45000]	Loss: 3.8550	LR: 0.000010
Training Epoch: 4 [2320/45000]	Loss: 3.8525	LR: 0.000010
Training Epoch: 4 [2336/45000]	Loss: 3.6647	LR: 0.000010
Training Epoch: 4 [2352/45000]	Loss: 3.7079	LR: 0.000010
Training Epoch: 4 [2368/45000]	Loss: 3.7819	LR: 0.000010
Training Epoch: 4 [2384/45000]	Loss: 3.6827	LR: 0.000010
Training Epoch: 4 [2400/45000]	Loss: 3.7995	LR: 0.000010
Training Epoch: 4 [2416/45000]	Loss: 3.8147	LR: 0.000010
Training Epoch: 4 [2432/45000]	Loss: 3.8041	LR: 0.000010
Training Epoch: 4 [2448/45000]	Loss: 3.7642	LR: 0.000010
Training Epoch: 4 [2464/45000]	Loss: 3.7419	LR: 0.000010
Training Epoch: 4 [2480/45000]	Loss: 3.7686	LR: 0.000010
Training Epoch: 4 [2496/45000]	Loss: 3.8753	LR: 0.000010
Training Epoch: 4 [2512/45000]	Loss: 3.8604	LR: 0.000010
Training Epoch: 4 [2528/45000]	Loss: 3.7495	LR: 0.000010
Training Epoch: 4 [2544/45000]	

Training Epoch: 4 [4560/45000]	Loss: 3.8540	LR: 0.000010
Training Epoch: 4 [4576/45000]	Loss: 3.7653	LR: 0.000010
Training Epoch: 4 [4592/45000]	Loss: 3.7507	LR: 0.000010
Training Epoch: 4 [4608/45000]	Loss: 3.6676	LR: 0.000010
Training Epoch: 4 [4624/45000]	Loss: 3.7224	LR: 0.000010
Training Epoch: 4 [4640/45000]	Loss: 3.7452	LR: 0.000010
Training Epoch: 4 [4656/45000]	Loss: 3.7591	LR: 0.000010
Training Epoch: 4 [4672/45000]	Loss: 3.7407	LR: 0.000010
Training Epoch: 4 [4688/45000]	Loss: 3.8202	LR: 0.000010
Training Epoch: 4 [4704/45000]	Loss: 3.6663	LR: 0.000010
Training Epoch: 4 [4720/45000]	Loss: 3.7411	LR: 0.000010
Training Epoch: 4 [4736/45000]	Loss: 3.6545	LR: 0.000010
Training Epoch: 4 [4752/45000]	Loss: 3.6572	LR: 0.000010
Training Epoch: 4 [4768/45000]	Loss: 3.8032	LR: 0.000010
Training Epoch: 4 [4784/45000]	Loss: 3.7601	LR: 0.000010
Training Epoch: 4 [4800/45000]	Loss: 3.6859	LR: 0.000010
Training Epoch: 4 [4816/45000]	Loss: 3.7074	LR: 0.000010
Training Epoch: 4 [4832/45000]	

Training Epoch: 4 [6848/45000]	Loss: 3.7772	LR: 0.000010
Training Epoch: 4 [6864/45000]	Loss: 3.6900	LR: 0.000010
Training Epoch: 4 [6880/45000]	Loss: 3.7608	LR: 0.000010
Training Epoch: 4 [6896/45000]	Loss: 3.7645	LR: 0.000010
Training Epoch: 4 [6912/45000]	Loss: 3.7133	LR: 0.000010
Training Epoch: 4 [6928/45000]	Loss: 3.7122	LR: 0.000010
Training Epoch: 4 [6944/45000]	Loss: 3.7668	LR: 0.000010
Training Epoch: 4 [6960/45000]	Loss: 3.7831	LR: 0.000010
Training Epoch: 4 [6976/45000]	Loss: 3.9332	LR: 0.000010
Training Epoch: 4 [6992/45000]	Loss: 3.7050	LR: 0.000010
Training Epoch: 4 [7008/45000]	Loss: 3.6058	LR: 0.000010
Training Epoch: 4 [7024/45000]	Loss: 3.6509	LR: 0.000010
Training Epoch: 4 [7040/45000]	Loss: 3.8079	LR: 0.000010
Training Epoch: 4 [7056/45000]	Loss: 3.7766	LR: 0.000010
Training Epoch: 4 [7072/45000]	Loss: 3.6859	LR: 0.000010
Training Epoch: 4 [7088/45000]	Loss: 3.7767	LR: 0.000010
Training Epoch: 4 [7104/45000]	Loss: 3.7135	LR: 0.000010
Training Epoch: 4 [7120/45000]	

Training Epoch: 4 [9136/45000]	Loss: 3.7258	LR: 0.000010
Training Epoch: 4 [9152/45000]	Loss: 3.8331	LR: 0.000010
Training Epoch: 4 [9168/45000]	Loss: 3.7882	LR: 0.000010
Training Epoch: 4 [9184/45000]	Loss: 3.7677	LR: 0.000010
Training Epoch: 4 [9200/45000]	Loss: 3.7325	LR: 0.000010
Training Epoch: 4 [9216/45000]	Loss: 3.8087	LR: 0.000010
Training Epoch: 4 [9232/45000]	Loss: 3.7956	LR: 0.000010
Training Epoch: 4 [9248/45000]	Loss: 3.7199	LR: 0.000010
Training Epoch: 4 [9264/45000]	Loss: 3.7916	LR: 0.000010
Training Epoch: 4 [9280/45000]	Loss: 3.6539	LR: 0.000010
Training Epoch: 4 [9296/45000]	Loss: 3.7435	LR: 0.000010
Training Epoch: 4 [9312/45000]	Loss: 3.7041	LR: 0.000010
Training Epoch: 4 [9328/45000]	Loss: 3.8245	LR: 0.000010
Training Epoch: 4 [9344/45000]	Loss: 3.6994	LR: 0.000010
Training Epoch: 4 [9360/45000]	Loss: 3.7495	LR: 0.000010
Training Epoch: 4 [9376/45000]	Loss: 3.7602	LR: 0.000010
Training Epoch: 4 [9392/45000]	Loss: 3.6992	LR: 0.000010
Training Epoch: 4 [9408/45000]	

...

Training Epoch: 4 [34400/45000]	Loss: 3.5459	LR: 0.000010
Training Epoch: 4 [34416/45000]	Loss: 3.5484	LR: 0.000010
Training Epoch: 4 [34432/45000]	Loss: 3.6376	LR: 0.000010
Training Epoch: 4 [34448/45000]	Loss: 3.5899	LR: 0.000010
Training Epoch: 4 [34464/45000]	Loss: 3.6140	LR: 0.000010
Training Epoch: 4 [34480/45000]	Loss: 3.5030	LR: 0.000010
Training Epoch: 4 [34496/45000]	Loss: 3.6444	LR: 0.000010
Training Epoch: 4 [34512/45000]	Loss: 3.6526	LR: 0.000010
Training Epoch: 4 [34528/45000]	Loss: 3.5222	LR: 0.000010
Training Epoch: 4 [34544/45000]	Loss: 3.5230	LR: 0.000010
Training Epoch: 4 [34560/45000]	Loss: 3.5228	LR: 0.000010
Training Epoch: 4 [34576/45000]	Loss: 3.5262	LR: 0.000010
Training Epoch: 4 [34592/45000]	Loss: 3.6215	LR: 0.000010
Training Epoch: 4 [34608/45000]	Loss: 3.5060	LR: 0.000010
Training Epoch: 4 [34624/45000]	Loss: 3.6647	LR: 0.000010
Training Epoch: 4 [34640/45000]	Loss: 3.7206	LR: 0.000010
Training Epoch: 4 [34656/45000]	Loss: 3.4892	LR: 0.000010
Training Epoch

Training Epoch: 4 [36656/45000]	Loss: 3.5415	LR: 0.000010
Training Epoch: 4 [36672/45000]	Loss: 3.6400	LR: 0.000010
Training Epoch: 4 [36688/45000]	Loss: 3.5446	LR: 0.000010
Training Epoch: 4 [36704/45000]	Loss: 3.5552	LR: 0.000010
Training Epoch: 4 [36720/45000]	Loss: 3.5592	LR: 0.000010
Training Epoch: 4 [36736/45000]	Loss: 3.5459	LR: 0.000010
Training Epoch: 4 [36752/45000]	Loss: 3.4974	LR: 0.000010
Training Epoch: 4 [36768/45000]	Loss: 3.5568	LR: 0.000010
Training Epoch: 4 [36784/45000]	Loss: 3.5595	LR: 0.000010
Training Epoch: 4 [36800/45000]	Loss: 3.6150	LR: 0.000010
Training Epoch: 4 [36816/45000]	Loss: 3.5916	LR: 0.000010
Training Epoch: 4 [36832/45000]	Loss: 3.4926	LR: 0.000010
Training Epoch: 4 [36848/45000]	Loss: 3.6165	LR: 0.000010
Training Epoch: 4 [36864/45000]	Loss: 3.4956	LR: 0.000010
Training Epoch: 4 [36880/45000]	Loss: 3.5915	LR: 0.000010
Training Epoch: 4 [36896/45000]	Loss: 3.4808	LR: 0.000010
Training Epoch: 4 [36912/45000]	Loss: 3.6436	LR: 0.000010
Training Epoch

Training Epoch: 4 [38912/45000]	Loss: 3.6490	LR: 0.000010
Training Epoch: 4 [38928/45000]	Loss: 3.6302	LR: 0.000010
Training Epoch: 4 [38944/45000]	Loss: 3.6285	LR: 0.000010
Training Epoch: 4 [38960/45000]	Loss: 3.6876	LR: 0.000010
Training Epoch: 4 [38976/45000]	Loss: 3.6338	LR: 0.000010
Training Epoch: 4 [38992/45000]	Loss: 3.6233	LR: 0.000010
Training Epoch: 4 [39008/45000]	Loss: 3.6359	LR: 0.000010
Training Epoch: 4 [39024/45000]	Loss: 3.5925	LR: 0.000010
Training Epoch: 4 [39040/45000]	Loss: 3.5633	LR: 0.000010
Training Epoch: 4 [39056/45000]	Loss: 3.7182	LR: 0.000010
Training Epoch: 4 [39072/45000]	Loss: 3.5760	LR: 0.000010
Training Epoch: 4 [39088/45000]	Loss: 3.6582	LR: 0.000010
Training Epoch: 4 [39104/45000]	Loss: 3.5935	LR: 0.000010
Training Epoch: 4 [39120/45000]	Loss: 3.5494	LR: 0.000010
Training Epoch: 4 [39136/45000]	Loss: 3.6019	LR: 0.000010
Training Epoch: 4 [39152/45000]	Loss: 3.5707	LR: 0.000010
Training Epoch: 4 [39168/45000]	Loss: 3.4808	LR: 0.000010
Training Epoch

Training Epoch: 4 [41168/45000]	Loss: 3.6761	LR: 0.000010
Training Epoch: 4 [41184/45000]	Loss: 3.6270	LR: 0.000010
Training Epoch: 4 [41200/45000]	Loss: 3.4579	LR: 0.000010
Training Epoch: 4 [41216/45000]	Loss: 3.7717	LR: 0.000010
Training Epoch: 4 [41232/45000]	Loss: 3.5652	LR: 0.000010
Training Epoch: 4 [41248/45000]	Loss: 3.5322	LR: 0.000010
Training Epoch: 4 [41264/45000]	Loss: 3.5483	LR: 0.000010
Training Epoch: 4 [41280/45000]	Loss: 3.6224	LR: 0.000010
Training Epoch: 4 [41296/45000]	Loss: 3.6330	LR: 0.000010
Training Epoch: 4 [41312/45000]	Loss: 3.6143	LR: 0.000010
Training Epoch: 4 [41328/45000]	Loss: 3.5317	LR: 0.000010
Training Epoch: 4 [41344/45000]	Loss: 3.6651	LR: 0.000010
Training Epoch: 4 [41360/45000]	Loss: 3.5645	LR: 0.000010
Training Epoch: 4 [41376/45000]	Loss: 3.6364	LR: 0.000010
Training Epoch: 4 [41392/45000]	Loss: 3.6590	LR: 0.000010
Training Epoch: 4 [41408/45000]	Loss: 3.5860	LR: 0.000010
Training Epoch: 4 [41424/45000]	Loss: 3.6267	LR: 0.000010
Training Epoch

Training Epoch: 4 [43424/45000]	Loss: 3.5831	LR: 0.000010
Training Epoch: 4 [43440/45000]	Loss: 3.5875	LR: 0.000010
Training Epoch: 4 [43456/45000]	Loss: 3.5891	LR: 0.000010
Training Epoch: 4 [43472/45000]	Loss: 3.5995	LR: 0.000010
Training Epoch: 4 [43488/45000]	Loss: 3.5461	LR: 0.000010
Training Epoch: 4 [43504/45000]	Loss: 3.5010	LR: 0.000010
Training Epoch: 4 [43520/45000]	Loss: 3.5037	LR: 0.000010
Training Epoch: 4 [43536/45000]	Loss: 3.5309	LR: 0.000010
Training Epoch: 4 [43552/45000]	Loss: 3.5640	LR: 0.000010
Training Epoch: 4 [43568/45000]	Loss: 3.5880	LR: 0.000010
Training Epoch: 4 [43584/45000]	Loss: 3.6589	LR: 0.000010
Training Epoch: 4 [43600/45000]	Loss: 3.5128	LR: 0.000010
Training Epoch: 4 [43616/45000]	Loss: 3.5172	LR: 0.000010
Training Epoch: 4 [43632/45000]	Loss: 3.5402	LR: 0.000010
Training Epoch: 4 [43648/45000]	Loss: 3.6924	LR: 0.000010
Training Epoch: 4 [43664/45000]	Loss: 3.6409	LR: 0.000010
Training Epoch: 4 [43680/45000]	Loss: 3.6327	LR: 0.000010
Training Epoch

In [ ]:
pickle.dump